In [1]:
import numpy as np
from scipy import signal
import torch
import json
import random
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.nn import Module
from collections import defaultdict

from GAIL.models.nets import Expert
from GAIL.models.gail import GAIL

from utilities import *
from utilitiesDL import *

In [2]:
jsonFileName = './inputJson/genData_TAR.json'
attackScheme = "BC"

f = open(jsonFileName)
inputJson = json.load(f)
dataPathBase = inputJson['dataPathBase']
dataType = inputJson['dataType']
padLen = inputJson['padLen']
dataWin = inputJson['dataWin']
cudaID = 5
f.close()

print("Datatype:", dataType)
if dataType == 'JAR':
    fs = 320
    nSubC = 30
    nRX = 3
    
    dSampFactor = 1
    activities = ['sitstill', 'falldown', 'liedown', 'standstill', 'walk', 'turn', 'stand', 'sit']

elif dataType == 'TAR':
    fs = 1000 # 1 kHz
    nSubC = 30
    nRX = 3
    
    dSampFactor = 10
    padLen = 9
    activities = ['bed', 'fall', 'run', 'sitdown', 'standup', 'walk']
    # activities = ['fall', 'pickup', 'run', 'sitdown', 'standup', 'walk']

torch.set_num_threads(1)
if cudaID >= 0:
    device = torch.device("cuda:"+str(cudaID))
    cudaAvbl = True
else:
    device = torch.device("cpu")
    cudaAvbl = False

dataPath = dataPathBase + "HAR_" + dataType + "/"
if dataType == 'JAR':
    dataPath = dataPath + "dSamp_" + str(dSampFactor) + "/"
elif dataType == 'TAR':
    if dataWin:
        padLen = 250
        dataPath = dataPath + "win_dSamp_" + str(dSampFactor) + "_pad_" + str(padLen) + "/"
        # dataPath = dataPath + "win_dSamp_" + str(dSampFactor) + "_" + str(attackScheme) + "/"
    else:
        dataPath = dataPath + "noWin_dSamp_" + str(dSampFactor) + "_pad_" + str(padLen) + "/"
print('dataPath:', dataPath)

Datatype: TAR
dataPath: /project/iarpa/wifiHAR/HAR_TAR/noWin_dSamp_10_pad_9/


In [7]:
inputJsonFileName = 'test'
LSTMModelDir = './savedModels/selected/'

inputJsonFile = open("./inputJson/varGAIL/" + inputJsonFileName + ".json", "r")
inputJson = json.load(inputJsonFile)
LSTMTargetModelName = inputJson['LSTMTargetModelName']
LSTMSurroModelName = inputJson['LSTMSurroModelName']
noiseAmpRatio = inputJson['noiseAmpRatio']
trDataList = inputJson['trDataList']
trExpDataRatio = inputJson['trExpDataRatio']
nHiddenGAIL = inputJson['nHiddenGAIL']
padLen = inputJson['padLen']
inputLenTime = inputJson['inputLenTime']
outputLenTime = inputJson['outputLenTime']
dSampFactor = inputJson['dSampFactor']
delayLen = inputJson['delayLen']
GAILTrainConfig = inputJson['trainConfig']
print(dSampFactor, padLen)

cudaID = 5
attackScheme = 'GAIL'

# padLen = 9

torch.set_num_threads(1)

if cudaID >= 0:
    device = torch.device("cuda:"+str(cudaID))
    cudaAvbl = True
else:
    device = torch.device("cpu")
    cudaAvbl = False
cudaLoc = 'cuda:' + str(cudaID)

dataType = LSTMTargetModelName.split('_')[0]
if dataType == 'TAR':
    fs = 1000 # 1 kHz
    nSubC = 30
    nRX = 3
    
    winLen = 1000
    activities = ['bed', 'fall', 'run', 'sitdown', 'standup', 'walk']
    # activities = ['bed', 'fall', 'run', 'sitdown', 'standup', 'walk']

    targetSingleAct = True
    testAct = ['bed', 'fall', 'run', 'sitdown', 'standup', 'walk']

print(LSTMSurroModelName, LSTMTargetModelName)
LSTMTargetType = LSTMTargetModelName.split('_')[2]
biDirTarget = (LSTMTargetType == 'BiLSTM')
LSTMSurroType = LSTMSurroModelName.split('_')[1]
biDirSurro = (LSTMSurroType == 'BiLSTM')
nHiddenTarget = int(LSTMTargetModelName.split('_')[4])
nHiddenSurro = int(LSTMSurroModelName.split('_')[3])
# nLayer = int(LSTMModelName.split('_')[7])
targetWin = (LSTMTargetModelName.split('_')[5] == 'win')
saveFileName = LSTMSurroModelName + '_' + inputJsonFileName

# Load target/surro LSTM models
if targetWin:
    HARTargetNet = LSTMNet_TAR(nClasses=len(activities),\
                    input_size=nSubC*nRX,\
                    bidirectional=biDirTarget,\
                    hidden_size=nHiddenTarget,\
                    num_layers=1,\
                    seq_length=1000,\
                    device=device)
    dSampRatio = 5
else:
    HARTargetNet = VariableLSTMNet(nClasses=len(activities),\
                    input_size=nSubC*nRX,\
                    bidirectional=biDirTarget,\
                    hidden_size=nHiddenTarget,\
                    num_layers=1,\
                    device=device)
HARTargetNet.load_state_dict(torch.load(LSTMModelDir + LSTMTargetModelName + '.cpkt', map_location=cudaLoc))

HARSurroNet = VariableLSTMNet(nClasses=len(activities),\
                input_size=nSubC*nRX,\
                bidirectional=biDirSurro,\
                hidden_size=nHiddenSurro,\
                num_layers=1,\
                device=device)
HARSurroNet.load_state_dict(torch.load(LSTMModelDir + LSTMSurroModelName + '.cpkt', map_location=cudaLoc))

# HARNet.to(device)

# Load dataset labelled with FGM attack
FGMdatasetDir = '/project/iarpa/wifiHAR/HAR_' + dataType + '/noWin_dSamp_' + str(dSampFactor) + '_pad_' + str(padLen) + '_FGM/'
dataDict = {file:[] for file in activities}
# tsDataDict = {file:[] for file in activities}

trDataset = list()
tsDataset = list()
correct = 0.0
nData = 0.0
noiseAmpRatio = 1e-4
# activities = ['walk']
longestLenList = list()
for actIdx, activity in enumerate(activities):
    longestLen = 0
    dataDict[activity] = defaultdict(list)

    dataInputActFileName = FGMdatasetDir + 'input_' + LSTMSurroModelName + '_' + activity + '.npy'
    dataFGMActFileName = FGMdatasetDir + 'FGM_' + LSTMSurroModelName + '_' + activity + '.npy'

    datasetObs = np.load(dataInputActFileName, allow_pickle=True)
    datasetFGM = np.load(dataFGMActFileName, allow_pickle=True)
    dSampRatio = 1
    for (ob, FGM) in zip(datasetObs, datasetFGM):
        print(ob[::dSampRatio, :].shape, FGM.shape, end=' ')
        # print(ob.shape, FGM.shape, end=' ')
        dataDict[activity]['input'].append(ob[::dSampRatio, :])
        dataDict[activity]['FGM'].append(FGM)
    dataDict[activity]['label'] = actIdx*torch.ones(len(dataDict[activity]['input']), dtype=int, device=device)

    # for i in range(0, ob.shape[0], inputLenTime*dSampRatio):
    #     # print(i, end=' ')
    #     if i+inputLenTime*dSampRatio > ob.shape[0]:
    #         break
    #     dataDict[activity]['input'].append(ob[i:i+inputLenTime*dSampRatio:dSampRatio, :])
    #     dataDict[activity]['label'].append(FGM[(i//dSampRatio), :])
    # longestLenList.append(longestLen)
            
    print('activity:', activity, 'nData:', len(dataDict[activity]['input']))

2 25
TAR_BiLSTM_H_200_dS_10 TAR_target_BiLSTM_H_200_win
torch.Size([1296, 90]) torch.Size([255, 90]) torch.Size([1956, 90]) torch.Size([387, 90]) torch.Size([1926, 90]) torch.Size([381, 90]) torch.Size([1436, 90]) torch.Size([283, 90]) torch.Size([2176, 90]) torch.Size([431, 90]) torch.Size([1516, 90]) torch.Size([299, 90]) torch.Size([1536, 90]) torch.Size([303, 90]) torch.Size([1621, 90]) torch.Size([320, 90]) torch.Size([1381, 90]) torch.Size([272, 90]) torch.Size([1516, 90]) torch.Size([299, 90]) torch.Size([2111, 90]) torch.Size([418, 90]) torch.Size([1771, 90]) torch.Size([350, 90]) torch.Size([1686, 90]) torch.Size([333, 90]) torch.Size([1296, 90]) torch.Size([255, 90]) torch.Size([2041, 90]) torch.Size([404, 90]) torch.Size([2196, 90]) torch.Size([435, 90]) torch.Size([1956, 90]) torch.Size([387, 90]) torch.Size([2431, 90]) torch.Size([482, 90]) torch.Size([1651, 90]) torch.Size([326, 90]) torch.Size([1431, 90]) torch.Size([282, 90]) torch.Size([1381, 90]) torch.Size([272, 90])

In [8]:
class CSI_BCDataset(Dataset):
    def __init__(self, dataDict, device, normalize=False, nSubC=1, nRX=1, padLen=0):
        self.features = dataDict['input'] # using IQ sample value
        self.FGM = dataDict['FGM']
        self.labels = dataDict['label']
        self.device = device
        self.normalize = normalize
        self.nSubC = nSubC
        self.nRX = nRX
        self.padLen = padLen

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        data = torch.tensor(self.features[idx], device=self.device).float()
        # print('what', data.shape)
        data = data[self.padLen:, :]
        FGM = torch.tensor(self.FGM[idx], device=self.device).float()
        if self.normalize:
            data = data * torch.numel(data) /\
                (LA.norm(data) * self.nSubC * self.nRX)
            FGM = FGM * torch.numel(FGM) /\
                (LA.norm(FGM) * self.nSubC * self.nRX)
        # print(LA.norm(data))

        label = torch.tensor(self.labels[idx], device=self.device).long()
        return {'input': data, 'label': label, 'FGM': FGM}

In [9]:
trData = list()
tsData = list()
for actIndex, activity in enumerate(activities):
    dataset = CSI_BCDataset(dataDict[activity],\
                    device,\
                    normalize=True,\
                    nSubC=nSubC,\
                    nRX=nRX,\
                    padLen=0)
    if actIndex == 0:
        tsDataList = [0, 1, 3, 4, 7, 10, 11, 13, 14, 15, 21, 22, 24, 27, 28, 29, 30, 32, 39, 40,\
            41, 42, 43, 46, 49, 52, 53, 55, 59, 60, 61, 62, 63, 67, 68, 69, 70, 73, 76, 78]
        trDataList = list(set(range(len(dataset))) - set(tsDataList))

    trData.append(torch.utils.data.Subset(dataset, trDataList))
    tsData.append(torch.utils.data.Subset(dataset, tsDataList))
    print(activity, len(trDataList), len(tsDataList), len(dataset))

trDataset = torch.utils.data.ConcatDataset(trData) # concatenating dataset lists
tsDataset = torch.utils.data.ConcatDataset(tsData)
print("# of tr data:", len(trDataset), "# of ts data:", len(tsDataset))

batchSize = 1
trLoader = DataLoader(trDataset, batch_size=batchSize, shuffle=False, collate_fn=collate_fn_FGM)
tsLoader = DataLoader(tsDataset, batch_size=batchSize, shuffle=False, collate_fn=collate_fn_FGM)

bed 39 40 79
fall 39 40 79
run 39 40 80
sitdown 39 40 80
standup 39 40 79
walk 39 40 80
# of tr data: 234 # of ts data: 240


In [10]:
class LSTM_BC(nn.Module):
    def __init__(self, input_size, output_size, bidirectional, hidden_size, num_layers, seq_length, device):
        super(LSTM_BC, self).__init__()
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.output_size = output_size #output size
        self.bidirectional = bidirectional #flag indicating whether LSTM is bidirectional
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        self.device = device

        self.lstm = nn.LSTM(input_size=input_size,\
                            hidden_size=hidden_size,\
                            num_layers=num_layers,\
                            batch_first=True,\
                            bidirectional=bidirectional,\
                            device=self.device) #lstm
        if bidirectional:
            self.fc = nn.Linear(2*hidden_size*seq_length, output_size, device=self.device) #fully connected last layer
        else:
            self.fc = nn.Linear(hidden_size*seq_length, output_size, device=self.device) #fully connected last layer
        self.relu = nn.ReLU()

    def forward(self,x):
        if self.bidirectional:
            h_0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size).to(self.device) #hidden state
            c_0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size).to(self.device) #internal state
        else:
            h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device) #hidden state
            c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device) #internal state
            
        # Propagate input through LSTM
        hi, (hn, _) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hi = hi.reshape(hi.shape[0], -1) #reshaping the data for Dense layer next

        out = self.relu(hi)
        out = self.fc(out) #Final Output
        return out

In [32]:
LSTM_BCLoss = torch.nn.MSELoss()
bidirectional = True
nHidden = 200
maxPatience = 40
LR = 2e-4
nEpoch = 400

fromInit = False
print(LSTMSurroModelName)
HARNetSavePath = './savedModels/varGAIL/TAR_BiLSTM_H_200_dS_10_nH_200_dS_10_LR_1e-5_dLen_1_iL_5_BC.cpkt' 

print(len(trLoader), len(tsLoader))
HARNet_BC = LSTM_BC(input_size=nSubC*nRX,\
                output_size=nSubC*nRX,\
                bidirectional=bidirectional,\
                hidden_size=nHidden,\
                num_layers=1,\
                seq_length=5,\
                device=device)
HARNet_BC.to(device)

if fromInit:
    HARNet_BC.apply(init_weights)
else:
    HARNet_BC.load_state_dict(torch.load(HARNetSavePath))
opt = torch.optim.Adam(HARNet_BC.parameters(), lr=LR)

TAR_BiLSTM_H_200_dS_10
234 240


In [24]:
bestLoss = 10.0
patience = 0
fastConvg = True
# innerBatchSize = 100

torch.set_num_threads(1)
print('save model path:', HARNetSavePath)
for epoch in range(nEpoch):
    runningLoss = 0.0
    HARNet_BC.train()
    nTrDataBC = 0
    for trData in trLoader:
        trInput, trLabel, trFGM = trData

        trInputBatch = torch.empty((0, inputLenTime, nSubC*nRX)).to(device)
        trFGMBatch = torch.empty((0, nSubC*nRX)).to(device)
        for i in range(0, trFGM.shape[1], inputLenTime):
            # print(i, end=' ')
            trInputBatch = torch.cat((trInputBatch, trInput[:, i:i+inputLenTime, :]), 0)
            trFGMBatch = torch.cat((trFGMBatch, trFGM[:, i, :]), 0)
            nTrDataBC = nTrDataBC+1
        # print(trInputBatch.shape, trFGMBatch.shape)
        
        opt.zero_grad()
        trOutput = HARNet_BC(trInputBatch)
        trloss = LSTM_BCLoss(trOutput, trFGMBatch)
        trloss.backward()
        opt.step()
        runningLoss += trloss.item()

    avgTrLoss = runningLoss / nTrDataBC
    # accTrain = getAcc(trLoader, padLen, HARNet_BC, HARNet_BC, variableLen=True)

    runningLoss = 0.0
    nTsDataBC = 0
    for tsData in tsLoader:
        tsInput, tsLabel, tsFGM = tsData

        tsInputBatch = torch.empty((0, inputLenTime, nSubC*nRX)).to(device)
        tsFGMBatch = torch.empty((0, nSubC*nRX)).to(device)
        for i in range(0, tsFGM.shape[1], inputLenTime):
            tsInputBatch = torch.cat((tsInputBatch, tsInput[:, i:i+inputLenTime, :]), 0)
            tsFGMBatch = torch.cat((tsFGMBatch, tsFGM[:, i, :]), 0)
            nTsDataBC = nTsDataBC+1
        # print(trInputBatch.shape, tsFGMBatch.shape)

        opt.zero_grad()
        
        tsOutput = HARNet_BC(tsInputBatch)
        tsloss = LSTM_BCLoss(tsOutput, tsFGMBatch)
        runningLoss += tsloss.item()

    avgTsLoss = runningLoss / nTsDataBC

    print('Epoch: %d, trLoss: %.6f, tsLoss: %.6f'\
        % (epoch, avgTrLoss, avgTsLoss), end=' ')
    if bestLoss > avgTsLoss:
        bestLoss = avgTsLoss
        print('saving model', end=' ')
        torch.save(HARNet_BC.state_dict(), HARNetSavePath)  # saving model with best test accuracy
        patience = 0

    # print(trData.shape, trLabel.shape)
    # early stopping if model converges twice
    patience += 1
    if patience > maxPatience:
        if fastConvg:
            LR = LR/10
            opt = torch.optim.Adam(HARNet_BC.parameters(), lr=LR)
            HARNet_BC.load_state_dict(torch.load(HARNetSavePath))
            patience = 0
            fastConvg = False
            print('fast convergence ends', end=' ')
        else:
            break
    print(' ')
print('Training finished')

save model path: ./savedModels/varGAIL/TAR_BiLSTM_H_200_dS_10_nH_200_dS_10_LR_1e-5_dLen_1_iL_5_BC.cpkt


/tmp/ipykernel_1602887/1903631498.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(self.features[idx], device=self.device).float()
/tmp/ipykernel_1602887/1903631498.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  FGM = torch.tensor(self.FGM[idx], device=self.device).float()
/tmp/ipykernel_1602887/1903631498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], device=self.device).long()
/home/byk004/24-RT_adv_HAR/utilities.py:31: UserWarning: To copy construct from a tenso

Epoch: 0, trLoss: 0.068574, tsLoss: 0.072071 saving model  
Epoch: 1, trLoss: 0.068488, tsLoss: 0.072208  
Epoch: 2, trLoss: 0.068388, tsLoss: 0.072358  
Epoch: 3, trLoss: 0.068354, tsLoss: 0.072336  
Epoch: 4, trLoss: 0.068337, tsLoss: 0.072394  
Epoch: 5, trLoss: 0.068309, tsLoss: 0.072415  
Epoch: 6, trLoss: 0.068217, tsLoss: 0.072528  
Epoch: 7, trLoss: 0.068201, tsLoss: 0.072539  
Epoch: 8, trLoss: 0.068102, tsLoss: 0.072543  
Epoch: 9, trLoss: 0.068081, tsLoss: 0.072756  
Epoch: 10, trLoss: 0.068060, tsLoss: 0.072655  
Epoch: 11, trLoss: 0.068008, tsLoss: 0.072802  
Epoch: 12, trLoss: 0.067952, tsLoss: 0.072822  
Epoch: 13, trLoss: 0.067973, tsLoss: 0.072968  
Epoch: 14, trLoss: 0.067918, tsLoss: 0.072944  
Epoch: 15, trLoss: 0.067935, tsLoss: 0.072838  
Epoch: 16, trLoss: 0.067878, tsLoss: 0.072758  
Epoch: 17, trLoss: 0.067826, tsLoss: 0.072833  
Epoch: 18, trLoss: 0.067794, tsLoss: 0.072831  
Epoch: 19, trLoss: 0.067735, tsLoss: 0.072824  
Epoch: 20, trLoss: 0.067636, tsLoss: 

In [6]:
modelTargetName = 'TAR_BiLSTM_H_200_dS_10'
HARNetTargetSavePath = './savedModels/selected/' + modelTargetName + ".cpkt"

biDirTarget = True
HARNetTarget = VariableLSTMNet(nClasses=len(activities),\
                input_size=nSubC*nRX,\
                bidirectional=biDirTarget,\
                hidden_size=nHiddenTarget,\
                num_layers=1,\
                device=device)
HARNetTarget.to(device)
HARNetTarget.load_state_dict(torch.load(HARNetTargetSavePath))
HARNetTarget.eval()

noiseAmpRatio = 0.0
accTest = getAcc(tsLoader,\
                padLen,\
                HARNetTarget,\
                HARNetTarget,\
                variableLen=True,\
                noiseAmpRatio=noiseAmpRatio,\
                noiseType='FGM')
print(accTest)

/tmp/ipykernel_1592961/1903631498.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(self.features[idx], device=self.device).float()
/tmp/ipykernel_1592961/1903631498.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  FGM = torch.tensor(self.FGM[idx], device=self.device).float()
/tmp/ipykernel_1592961/1903631498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], device=self.device).long()
/home/byk004/24-RT_adv_HAR/utilities.py:31: UserWarning: To copy construct from a tenso

0.9708333333333333


In [23]:
nHiddenSurro = 200
modelSurroName = 'TAR_BiLSTM_H_200_dS_10_nH_200_dS_10_LR_1e-5_dLen_1_iL_5_BC'
HARNetSurroSavePath = './savedModels/selected/varGAIL/' + modelSurroName + ".cpkt"
biDirSurro = True

nHiddenTarget = 200
# modelTargetName = 'TAR_BiLSTM_H_200_dS_10'
modelTargetName = 'TAR_target_BiLSTM_H_200_win'
HARNetTargetSavePath = './savedModels/selected/' + modelTargetName + ".cpkt"
biDirTarget = True

HARNetSurro = LSTM_BC(input_size=nSubC*nRX,\
                output_size=nSubC*nRX,\
                bidirectional=biDirSurro,\
                hidden_size=nHiddenSurro,\
                num_layers=1,\
                seq_length=5,\
                device=device)
HARNetSurro.to(device)
HARNetSurro.load_state_dict(torch.load(HARNetSurroSavePath))
HARNetSurro.eval()

# HARNetTarget = VariableLSTMNet(nClasses=len(activities),\
#                 input_size=nSubC*nRX,\
#                 bidirectional=biDirTarget,\
#                 hidden_size=nHiddenTarget,\
#                 num_layers=1,\
#                 device=device)

HARNetTarget = LSTMNet_TAR(nClasses=len(activities),\
                input_size=nSubC*nRX,\
                bidirectional=biDirTarget,\
                hidden_size=nHiddenTarget,\
                num_layers=1,\
                seq_length=1000,\
                device=device)

HARNetTarget.to(device)
HARNetTarget.load_state_dict(torch.load(HARNetTargetSavePath))
HARNetTarget.eval()

LSTMNet_TAR(
  (lstm): LSTM(90, 200, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=400000, out_features=6, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [19]:
# noiseAmpRatioList = [1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 0.1, 0.2, 0.5, 1, 2, 5, 10]
noiseAmpRatioList = [1e-1]

correct = [0. for _ in noiseAmpRatioList]
padLen = 5
dSampRatio = 5
for tsData in tsLoader:
    tsInputData, tsLabel, tsFGM = tsData
    tsInput = tsInputData[:, ::dSampRatio, :]
    tsInput = tsInput * torch.numel(tsInput) / (LA.norm(tsInput) * nSubC * nRX)

    # print(tsInput.shape, tsFGM.shape)
    tsInputBatch = torch.empty((0, inputLenTime, nSubC*nRX)).to(device)
    for i in range(0, tsInput.shape[1]-inputLenTime):
        tsInputBatch = torch.cat((tsInputBatch, tsInput[:, i:i+inputLenTime, :]), 0)
    tsSurroOutput = HARNetSurro(tsInputBatch)
    # tsTargetInput = tsInput[:, padLen:, :] + tsSurroOutput
    # for noiseAmpIndex, noiseAmpRatio in enumerate(noiseAmpRatioList):
        # pred, label = getPredsGAIL(tsInput[:, padLen:, :], torch.unsqueeze(tsSurroOutput, 0), tsLabel,\
        #                             HARNetTarget, noiseAmpRatio)
        
        # correct[noiseAmpIndex] += (pred == label)

/tmp/ipykernel_1664864/1903631498.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(self.features[idx], device=self.device).float()
/tmp/ipykernel_1664864/1903631498.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  FGM = torch.tensor(self.FGM[idx], device=self.device).float()
/tmp/ipykernel_1664864/1903631498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], device=self.device).long()
/home/byk004/24-RT_adv_HAR/utilities.py:31: UserWarning: To copy construct from a tenso

In [45]:
LSTMSurroModelName

'TAR_BiLSTM_H_200_dS_10'

In [46]:
batchSize = 1
slideLen = 200
seqLen = 1000
padLen = 25
noiseAmpRatio = 0.1
dSampTarget = 2
attackScheme = 'BC'

for actInd, activity in enumerate(activities):
    dataInput_fileName = "/project/iarpa/wifiHAR/HAR_TAR/win_dSamp_" +\
        str(dSampTarget)+ "_" + attackScheme + "/input_" + LSTMSurroModelName + "_" + activity
    dataFGM_fileName = "/project/iarpa/wifiHAR/HAR_TAR/win_dSamp_" +\
        str(dSampTarget)+ "_" + attackScheme + "/" + attackScheme + "_" + LSTMSurroModelName + "_" + activity

    print(activity)
    inputList = []
    actsList = []


    tsDatasetAct = CSI_BCDataset(dataDict[activity], device, padLen=0)
    tsDataset = list()
    tsDataset.append(torch.utils.data.Subset(tsDatasetAct, tsDataList))
    tsLoaderAct = DataLoader(torch.utils.data.ConcatDataset(tsDataset), batch_size=batchSize, shuffle=False, collate_fn=collate_fn_FGM)

    # print(len(tsLoaderAct))
    for tsData in tsLoaderAct:
        tsInputData, tsLabel, tsFGM = tsData
        tsDataWoPad = tsInputData[:, padLen:, :]
        tsInput = tsInputData[:, ::dSampRatio, :]
        tsInput = tsInput * torch.numel(tsInput) / (LA.norm(tsInput) * nSubC * nRX)

        tsInputBatch = torch.empty((0, inputLenTime, nSubC*nRX)).to(device)
        for i in range(0, tsInput.shape[1]-inputLenTime):
            tsInputBatch = torch.cat((tsInputBatch, tsInput[:, i:i+inputLenTime, :]), 0)
        tsSurroOutput = HARNetSurro(tsInputBatch)
        # print(tsInputData.shape, tsInputBatch.shape, tsSurroOutput.shape)

        actsDataUSamp = torch.Tensor().to(device)
        for uSampIdx, tIdx in enumerate(range(0, tsDataWoPad.shape[1], dSampRatio)):
            actsTimeSamp = tsSurroOutput[uSampIdx, :]
            # print(actsTimeSamp.repeat(1, 5, 1).shape)
            actsDataUSamp = torch.cat((actsDataUSamp, actsTimeSamp.repeat(1, 5, 1)), dim=1)
        # print(actsDataUSamp.shape)

        for tIdx in range(0, tsDataWoPad.shape[1], slideLen):
            inputWin = tsDataWoPad[:, tIdx:tIdx+seqLen, :]
            actsWin = actsDataUSamp[:, tIdx:tIdx+seqLen, :]
            if inputWin.shape[1] < seqLen:
                continue
            # print(inputWin.shape, actsWin.shape)

            noiseAmp = LA.norm(torch.flatten(inputWin)) * noiseAmpRatio
            noiseFlatten = torch.flatten(actsWin)
            noiseNormalized = torch.mul(torch.div(noiseFlatten,\
                LA.norm(noiseFlatten)), noiseAmp)
            actsWin = noiseNormalized.view(inputWin.shape)

            inputList.append(torch.squeeze(inputWin).cpu().detach())
            actsList.append(torch.squeeze(actsWin).cpu().detach())

    np.save(dataInput_fileName, np.array(inputList, dtype=object), allow_pickle=True)
    np.save(dataFGM_fileName, np.array(actsList, dtype=object), allow_pickle=True)

bed
40


/tmp/ipykernel_1664864/1903631498.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(self.features[idx], device=self.device).float()
/tmp/ipykernel_1664864/1903631498.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  FGM = torch.tensor(self.FGM[idx], device=self.device).float()
/tmp/ipykernel_1664864/1903631498.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.labels[idx], device=self.device).long()
/home/byk004/24-RT_adv_HAR/utilities.py:31: UserWarning: To copy construct from a tenso

fall
40
run
40
